In [37]:
import os
import xmltodict
import pandas as pd
from pprint import pprint
from utils.tui import Progress
from utils.lexical import LexicalProcessing

BASE_DIR = '../data/corpora/trainset'
LP = LexicalProcessing()

In [2]:
def get_count():
    count = 0
    for product in os.listdir(BASE_DIR):
        corpus = {}

        product_path = '{}/{}'.format(BASE_DIR, product)
        for rank in os.listdir(product_path):  
            rank_path = '{}/{}'.format(product_path, rank)
            fls = os.listdir(rank_path)
            fls = [ x for x in fls if '.xml' in x ]
            count += len(fls)
    return count

In [3]:
def get_corpora():
    corpora_kabum = {}
    errors = []
    count = get_count()
    P = Progress(count, '')
    for product in os.listdir(BASE_DIR):
        corpus = {}

        product_path = '{}/{}'.format(BASE_DIR, product)
        for rank in os.listdir(product_path):  
            rank_path = '{}/{}'.format(product_path, rank)
            fls = os.listdir(rank_path)
            fls = [ x for x in fls if '.xml' in x ]

            reviews = []
            for fl_name in fls:
                fl_path = '{}/{}'.format(rank_path, fl_name)
                with open(fl_path) as fl_:
                    try:
                        r = xmltodict.parse(fl_.read())['review']
                        if not r['opinion']:
                            raise Exception(fl_path)
                        reviews.append(r)
                    except:
                        errors.append(fl_path)
                P.progressStep()
            corpus[rank] = reviews
        corpora_kabum[product] = corpus
    return (corpora_kabum, errors)

In [48]:
num_sents = []

def create_dataframe(corpora_kabum):
    dataset = dict(category=[], score=[], review=[])
    for cat in corpora_kabum.keys():
        for stars in corpora_kabum[cat].keys():
            for review in corpora_kabum[cat][stars]:
                dataset['category'].append(review['category']['@value'])
                dataset['score'].append(int(float(review['stars']['@value'])))
                num_sents.append(len(LP.tokenize_sentences(review['opinion'])))
                dataset['review'].append(review['opinion'])
    return pd.DataFrame(data=dataset)

In [49]:
#c_ = get_corpora()
df = create_dataframe(c_[0])

In [54]:
print('Quantidade de reviews: ', len(df))

Quantidade de reviews:  67010


In [55]:
import numpy

print('Quantidade de sentencas: ', numpy.sum(num_sents))

Quantidade de sentencas:  205145


In [56]:
print("Reviews com score 1: ", len(df[df.score == 1]))
print("Reviews com score 2: ", len(df[df.score == 2]))
print("Reviews com score 3: ", len(df[df.score == 3]))
print("Reviews com score 4: ", len(df[df.score == 4]))
print("Reviews com score 5: ", len(df[df.score == 5]))

Reviews com score 1:  2395
Reviews com score 2:  2892
Reviews com score 3:  8934
Reviews com score 4:  25769
Reviews com score 5:  26349
